In [74]:
import pandas as pd
import kagglehub
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# Download latest dataset and load it in Pandas 

In [ ]:
#Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/world-stock-prices-daily-updating")

print("Path to dataset files:", path)

100%|██████████| 11.9M/11.9M [00:00<00:00, 12.5MB/s]

Extracting files...


Path to dataset files: C:\Users\s3phi\.cache\kagglehub\datasets\nelgiriyewithana\world-stock-prices-daily-updating\versions\349


In [144]:
#Load Data in pandas
data_path = path+"\World-Stock-Prices-Dataset.csv"
data = pd.read_csv(data_path)

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\s3phi\AppData\Local\Temp\ipykernel_32760\3204793938.py:2: SyntaxWarning: invalid escape sequence '\W'
  data_path = path+"\World-Stock-Prices-Dataset.csv"


# Exploring Data

In [145]:
#Check data content
data.head(10)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Capital Gains
0,2025-05-15 00:00:00-04:00,6.500000,6.550000,6.290000,6.330000,9465400.0,0.0,0.0,peloton,PTON,fitness,usa,NaN
1,2025-05-15 00:00:00-04:00,117.019997,117.500000,114.010002,114.989998,43737300.0,0.0,0.0,amd,AMD,technology,usa,NaN
2,2025-05-15 00:00:00-04:00,123.839996,124.599998,121.580002,122.720001,32200.0,0.0,0.0,adidas,ADDYY,apparel,germany,NaN
3,2025-05-15 00:00:00-04:00,298.049988,300.929993,297.000000,299.549988,1955000.0,0.0,0.0,american express,AXP,finance,usa,NaN
4,2025-05-15 00:00:00-04:00,25.545000,25.545000,25.545000,25.545000,0.0,0.0,0.0,puma,PMMAF,apparel,germany,NaN
5,2025-05-15 00:00:00-04:00,356.890015,363.730011,356.679993,362.299988,5825600.0,0.0,0.0,visa,V,finance,usa,NaN
6,2025-05-15 00:00:00-04:00,399.350006,404.850006,399.350006,404.690002,2762700.0,0.0,0.0,adobe,ADBE,technology,usa,NaN
7,2025-05-15 00:00:00-04:00,61.380001,61.860001,61.200001,61.840000,1995200.0,0.0,0.0,unilever,UL,consumer goods,netherlands,NaN
8,2025-05-15 00:00:00-04:00,63.070000,65.750000,62.919998,64.260002,60707800.0,0.0,0.0,cisco,CSCO,technology,usa,NaN
9,2025-05-15 00:00:00-04:00,266.970001,269.519989,264.760010,267.489990,9212400.0,0.0,0.0,jpmorgan chase & co,JPM,finance,usa,NaN


In [146]:
#Check data types
data.dtypes

Date              object
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
Dividends        float64
Stock Splits     float64
Brand_Name        object
Ticker            object
Industry_Tag      object
Country           object
Capital Gains    float64
dtype: object

In [147]:
#Create DataFrame
df = pd.DataFrame(data)

In [148]:
#Filter Brand_Name by apple and filter columns by Date, Close, Brand_Name
df_apple = df.loc[df["Brand_Name"] == "apple", ["Date", "Close", "Brand_Name"]]
display(df_apple)

,Date,Close,Brand_Name
35,2025-05-15 00:00:00-04:00,211.449997,apple
69,2025-05-14 00:00:00-04:00,212.330002,apple
159,2025-05-13 00:00:00-04:00,212.929993,apple
180,2025-05-12 00:00:00-04:00,210.789993,apple
269,2025-05-05 00:00:00-04:00,198.889999,apple
...,...,...,...
307513,2000-01-07 00:00:00-05:00,0.754065,apple
307561,2000-01-06 00:00:00-05:00,0.719961,apple
307597,2000-01-05 00:00:00-05:00,0.788168,apple
307626,2000-01-04 00:00:00-05:00,0.776801,apple


In [149]:
df_apple.dtypes

Date           object
Close         float64
Brand_Name     object
dtype: object

In [ ]:
# try:
#     df_apple['Date'] = pd.to_datetime(df_apple['Date'], utc=True)  #Parsing into UTC
# except Exception as e:
#     print("Error converting 'Date' column:", e)

# df_apple.head(10)

,Date,Close,Brand_Name
35,2025-05-15 04:00:00+00:00,211.449997,apple
69,2025-05-14 04:00:00+00:00,212.330002,apple
159,2025-05-13 04:00:00+00:00,212.929993,apple
180,2025-05-12 04:00:00+00:00,210.789993,apple
269,2025-05-05 04:00:00+00:00,198.889999,apple
300,2025-05-02 04:00:00+00:00,205.350006,apple
395,2025-05-02 04:00:00+00:00,205.350006,apple
430,2025-05-01 04:00:00+00:00,213.320007,apple
518,2025-04-30 04:00:00+00:00,212.500000,apple
545,2025-04-29 04:00:00+00:00,211.210007,apple


In [150]:
df_apple_preproc = df_apple.rename(columns={"Date": "timestamp", "Close": "target", "Brand_Name": "item_id"})
df_apple_preproc.head(5)

,timestamp,target,item_id
35,2025-05-15 00:00:00-04:00,211.449997,apple
69,2025-05-14 00:00:00-04:00,212.330002,apple
159,2025-05-13 00:00:00-04:00,212.929993,apple
180,2025-05-12 00:00:00-04:00,210.789993,apple
269,2025-05-05 00:00:00-04:00,198.889999,apple


In [158]:
df_apple_preproc["item_id"] = df_apple_preproc['item_id'].astype("string")
df_apple_preproc["timestamp"] = df_apple_preproc['timestamp'].astype("string")
df_apple_preproc.dtypes

timestamp    string[python]
target              float64
item_id      string[python]
dtype: object

In [ ]:
timecut = df_apple_preproc["timestamp"].str.slice(stop=10) #Cut hh:mm:ss and timezone
df_apple_preproc.head(10)

,timestamp,target,item_id
35,2025-05-15,211.449997,apple
69,2025-05-14,212.330002,apple
159,2025-05-13,212.929993,apple
180,2025-05-12,210.789993,apple
269,2025-05-05,198.889999,apple
300,2025-05-02,205.350006,apple
395,2025-05-02,205.350006,apple
430,2025-05-01,213.320007,apple
518,2025-04-30,212.500000,apple
545,2025-04-29,211.210007,apple


In [ ]:
df_apple_preproc["timestamp"] = pd.to_datetime(timecut) #convert string into datetime64
df_apple_preproc.dtypes

timestamp    datetime64[ns]
target              float64
item_id      string[python]
dtype: object

In [166]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

load_data = TimeSeriesDataFrame.from_data_frame(
    df_apple_preproc,
    id_column="item_id",
    timestamp_column="timestamp"
)
load_data.head()

target
item_id timestamp             
apple   2025-05-15  211.449997
        2025-05-14  212.330002
        2025-05-13  212.929993
        2025-05-12  210.789993
        2025-05-05  198.889999

In [167]:
prediction_length = 10
train_data, test_data = load_data.train_test_split(prediction_length)

predictor = TimeSeriesPredictor(prediction_length=prediction_length, freq="w").fit(
    train_data, presets="bolt_small",
)

Sorting the dataframe index before generating the train/test split.
c:\workspace\CrimeMap\CrimeMap_venv\Lib\site-packages\autogluon\timeseries\predictor.py:198: FutureWarning: 'w' is deprecated and will be removed in a future version, please use 'W' instead.
  offset = pd.tseries.frequencies.to_offset(self.freq)
Frequency 'w' stored as 'W-SUN'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\workspace\CrimeMap\AutogluonModels\ag-20250517_112447'
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.12.1
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.37 GB / 15.70 GB (8.7%)
Disk Space Avail:   53.95 GB / 475.50 GB (11.3%)
Setting presets to: bolt_small

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'W-SUN',
 'hyperparameters': {'Chronos': {'model_path': 'bolt_small'}},
 'known_covariates_n

In [169]:
predictions = predictor.predict(train_data)
predictor.plot(
    data=data,
    predictions=predictions,
    item_ids=data.item_ids[:2],
    max_history_length=200,
);

data with frequency 'IRREG' has been resampled to frequency 'W-SUN'.


ValueError: Trainer has no fit models that can predict.

# Test

In [174]:
df_apple_preproc.to_csv('apple-out.csv', index=False)

In [182]:
data = TimeSeriesDataFrame.from_path(
    "worldstock-amd.csv"
)
data.head()

target
item_id timestamp             
amd     2025-05-14  117.720001
        2025-05-13  112.459999
        2025-05-12  108.120003
        2025-05-05  100.589996
        2025-05-02   98.800003

In [183]:
prediction_length = 20
train_data, test_data = data.train_test_split(prediction_length)

predictor = TimeSeriesPredictor(prediction_length=prediction_length, freq="w").fit(
    train_data, presets="bolt_small",
)

Sorting the dataframe index before generating the train/test split.
c:\workspace\CrimeMap\CrimeMap_venv\Lib\site-packages\autogluon\timeseries\predictor.py:198: FutureWarning: 'w' is deprecated and will be removed in a future version, please use 'W' instead.
  offset = pd.tseries.frequencies.to_offset(self.freq)
Frequency 'w' stored as 'W-SUN'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\workspace\CrimeMap\AutogluonModels\ag-20250517_120046'
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.12.1
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       1.57 GB / 15.70 GB (10.0%)
Disk Space Avail:   52.94 GB / 475.50 GB (11.1%)
Setting presets to: bolt_small

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'W-SUN',
 'hyperparameters': {'Chronos': {'model_path': 'bolt_small'}},
 'known_covariates_

In [187]:
predictions = predictor.predict(train_data)
predictor.plot(
    data=data,
    predictions=predictions,
    item_ids=data.item_ids[:2],
    max_history_length=200,
);

data with frequency 'IRREG' has been resampled to frequency 'W-SUN'.


ValueError: Trainer has no fit models that can predict.